In [1]:
import pandas as pd
amtrakData = pd.read_csv("amtrakdata.csv")
amtrakData = amtrakData.dropna(subset=['STNCODE'])

In [2]:
import pandas as pd


sample_df = amtrakData.sample(n=100000, random_state=42) 

sample_df = sample_df.drop(columns=['Actual Arrival Time', 'Actual Departure Time'])
sample_df['is_late'] = sample_df['Status'].apply(lambda x: 1 if x == "Late" else 0)

train_df = sample_df.sample(frac=0.8, random_state=42)
test_df = sample_df.drop(train_df.index)

print(f"Training set: {len(train_df)} rows")
print(f"Test set: {len(test_df)} rows")


Training set: 80000 rows
Test set: 20000 rows


In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder



encoded_features = pd.get_dummies(sample_df[['Station Code', 'Season', 'Weekday/Weekend']])
sample_df = pd.concat([sample_df, encoded_features], axis=1)

features = ['Year', 'Month', 'Day',
            'X', 'Y', 'Scheduled Arrival Hour',
            'Is Peak Hour', 'Is Holiday'] + list(encoded_features.columns)

X = sample_df[features]
y = sample_df['is_late']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
    n_estimators=100
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(classification_report(y_test, y_pred))


Accuracy: 74.02%
              precision    recall  f1-score   support

           0       0.68      0.34      0.46      6360
           1       0.75      0.92      0.83     13640

    accuracy                           0.74     20000
   macro avg       0.72      0.63      0.64     20000
weighted avg       0.73      0.74      0.71     20000



In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = sample_df[features]
y = sample_df['is_late']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(classification_report(y_test, y_pred))


Accuracy: 72.46%
              precision    recall  f1-score   support

           0       0.59      0.43      0.50      6360
           1       0.76      0.86      0.81     13640

    accuracy                           0.72     20000
   macro avg       0.68      0.64      0.65     20000
weighted avg       0.71      0.72      0.71     20000



In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



X = sample_df[features]
y = sample_df['is_late']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/50
2000/2000 [==============================] - 7s 2ms/step - loss: 0.6233 - accuracy: 0.6787 - val_loss: 0.5728 - val_accuracy: 0.7119
Epoch 2/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5785 - accuracy: 0.7104 - val_loss: 0.5664 - val_accuracy: 0.7240
Epoch 3/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5705 - accuracy: 0.7159 - val_loss: 0.5703 - val_accuracy: 0.7221
Epoch 4/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5675 - accuracy: 0.7186 - val_loss: 0.5644 - val_accuracy: 0.7232
Epoch 5/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5661 - accuracy: 0.7176 - val_loss: 0.5624 - val_accuracy: 0.7232
Epoch 6/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5639 - accuracy: 0.7195 - val_loss: 0.5623 - val_accuracy: 0.7266
Epoch 7/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5608 - accuracy: 0.7230 - val_loss: 0.5619 - val_accuracy:

In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

X = sample_df[features]
y = sample_df['is_late']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/50
2000/2000 [==============================] - 6s 2ms/step - loss: 0.6174 - accuracy: 0.6812 - val_loss: 0.5687 - val_accuracy: 0.7239
Epoch 2/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.5744 - accuracy: 0.7133 - val_loss: 0.5642 - val_accuracy: 0.7258
Epoch 3/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5698 - accuracy: 0.7152 - val_loss: 0.5620 - val_accuracy: 0.7264
Epoch 4/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5664 - accuracy: 0.7192 - val_loss: 0.5621 - val_accuracy: 0.7256
Epoch 5/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5649 - accuracy: 0.7186 - val_loss: 0.5611 - val_accuracy: 0.7260
Epoch 6/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.5633 - accuracy: 0.7190 - val_loss: 0.5604 - val_accuracy: 0.7254
Epoch 7/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.5617 - accuracy: 0.7214 - val_loss: 0.5606 - val_accuracy: